In [ ]:
from datetime import datetime, timezone, timedelta
import pandas as pd
import requests
import json

def get_time():
    now = datetime.now(timezone.utc)
    epoch = datetime(1970, 1, 1, tzinfo=timezone.utc) # use POSIX epoch
    posix_timestamp_micros = (now - epoch) // timedelta(microseconds=1)
    posix_timestamp_millis = posix_timestamp_micros // 1000 # or `/ 1e3` for float
    return posix_timestamp_millis

def request(data, sid):
    headers = {"content-type":'application/x-www-form-urlencoded'}
    resp = requests.post("https://www.matecat.com/?action={}&time={}-v0.6.1e,jid={},sid={}".format(get_time(), data['action'], jid, sid), headers=headers, data=data)
    #print(resp.text)
    return resp.json()


def setRevision(sid, orginal, review_data):
    data = {
    'action':'setRevision',
    'job':jid,
    'jpassword': password,
    'segment':sid,
    'original':orginal,
    'err_typing':0,
    'err_translation':0,
    'err_terminology':0,
    'err_language':0,
    'err_style':0
    }
    return request(data, sid)

def setTranslation(sid, file_id, translation):
    data ={
    'id_segment':sid,
    'id_job':jid,
    'id_first_file':file_id,
    'password':password,
    'status':'approved',
    'translation':translation,
    'id_translator':False,
    'errors':'',
    'chosen_suggestion_index':1,
    'autosave':False,
    'version':'',
    'propagate':True,
    'action':'setTranslation'
    }
    return request(data, sid)
    
def setCurrentSegment(sid):
    data = {
    'action':'setCurrentSegment',
    'password':password,
    'id_segment':sid,
    'id_job':jid
    }
    return request(data, sid)
    
def setContribution(sid, source, target, orginal, translation):
    data = {      
    'action':"setContribution",
    'id_job':jid,
    'source':orginal,
    'target':translation,
    'source_lang': source,
    'target_lang': target,
    'password':password,
    'id_translator':False,
    'private_translator':0,
    'id_customer':'unknown_customer',
    'private_customer':0
    }
    return request(data, sid)
    

def approve_range(start_segment, end_segment):
    if (end_segment - start_segment) < 0:
        print("Negative segment count. Aborting.")
        return
    
    print("Starting")
    data = {
    'action':'getSegments',
    'jid':jid,
    'password':password,
    'step':end_segment-start_segment,
    'segment':start_segment,
    'where':'after'
    }
    resp = request(data, start_segment)
    approved = []

    for file_id in resp['data']['files']:
        source_code = resp['data']['files'][file_id]['source_code']
        target_code = resp['data']['files'][file_id]['target_code']
        
        for segment in resp['data']['files'][file_id]['segments']:
            if not (int(segment['sid'])>=start_segment and int(segment['sid'])<=end_segment):
                continue
                
            print(segment['sid'], ' | ', segment['status'], ' | ', segment['segment'], ' | ', segment['translation'])
            
            if segment['status'].lower() == 'translated':
                if segment['segment'] in approved:
                    print("NO OP; AUTOPROPAGATED")
                else:
                    print("APPROVING...")
                    setRevision(segment['sid'], segment['translation'], {})
                    setTranslation(segment['sid'], file_id, segment['translation'])
                    setContribution(segment['sid'], source_code, target_code, segment['segment'], segment['translation'])
                    approved.append(segment['segment'])
            else:
                print("NO OP; ALREADY APPROVED")
                
    print("Finished")
    return segment



def get_credentials(url):
    return url.split('/')[-1].split('#')[0].split('-')


url = 'https://www.matecat.com/translate/blabla/EXAMPLEURL/jobid-password#<segmentnumber>'
jid, password = get_credentials(url)
start_segment = 100000000
end_segment = 100000100

segment = approve_range(start_segment, end_segment)
